<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/18_05_KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_KorNLI_%ED%92%80%EC%96%B4%EB%B3%B4%EA%B8%B0_(%EB%8B%A4%EC%A4%91_%ED%81%B4%EB%9E%98%EC%8A%A4_%EB%B6%84%EB%A5%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.0 MB/s eta 0:00:00


In [2]:
import transformers
transformers.__version__

'4.26.1'

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### 1. 데이터 로드 및 전처리

In [4]:
# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")
     

('snli_1.0_train.ko.tsv', <http.client.HTTPMessage at 0x7fa043da27c0>)

In [5]:
# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

('xnli.dev.ko.tsv', <http.client.HTTPMessage at 0x7fa14052e3a0>)

In [6]:
# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")


('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x7fa14052e820>)

In [7]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv('multinli.train.ko.tsv', sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [8]:
train_snli.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


In [9]:
train_snli.shape, train_xnli.shape, val_data.shape, test_data.shape

((550152, 3), (392702, 3), (2490, 3), (5010, 3))

In [11]:
# 결합후 섞기
train_data = pd.concat([train_snli, train_xnli])
train_data = train_data.sample(frac=1)

In [12]:
train_data.shape

(942854, 3)

In [15]:
# 빈값, 중복값 제거
def drop_na_and_duplicates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

In [16]:
train_data = drop_na_and_duplicates(train_data)
val_data = drop_na_and_duplicates(val_data)
test_data = drop_na_and_duplicates(test_data)

In [17]:
train_data.shape, val_data.shape, test_data.shape

((941814, 3), (2490, 3), (5010, 3))

### 2. 토큰화 및 임베딩

In [18]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [19]:
max_seq_len = 128

In [20]:
# 테스트트
sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]
print(sent1)
print(sent2)

네이비 아일랜드와 힐이라는 두 개의 만은 모두 쉽게 볼 수 있다.
올해 네이비 아일랜드와 힐은 쉽게 볼 수 없다.


In [23]:
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
print(encoding_result['input_ids'])
print('-'*50)
print(encoding_result['token_type_ids'])  # segmentation attention, 두 문장 분리를 알려줌
print('-'*50)
print(encoding_result['attention_mask'])  # 패딩 위치를 알려줌

[2, 29545, 10517, 2522, 1971, 2052, 23548, 864, 20750, 1038, 2073, 3702, 1311, 2318, 1164, 1295, 1513, 2062, 18, 3, 3753, 29545, 10517, 2522, 1971, 2073, 1311, 2318, 1164, 1295, 1415, 2062, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [25]:
# 위의 부분을 한꺼번에 실행할 수 있는 함수 정의
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, token_type_ids, attention_masks = [], [], []
  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)
    input_ids.append(encoding_result['input_ids'])
    token_type_ids.append(encoding_result['token_type_ids'])
    attention_masks.append(encoding_result['attention_mask'])

  input_ids = np.array(input_ids, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)

  return (input_ids, attention_masks, token_type_ids)

In [26]:
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len, tokenizer)

  0%|          | 0/941814 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 1309/941814 [00:01<14:32, 1078.39it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So th

In [27]:
# 최대 길이 128로 설정됨
# X_train 첫번째째 샘플 확인
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩 길이 :', len(input_id))
print('정수 인코딩 복원:', tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2 29545 10517  2522  1971  2052 23548   864 20750  1038  2073  3702
  1311  2318  1164  1295  1513  2062    18     3  3753 29545 10517  2522
  1971  2073  1311  2318  1164  1295  1415  2062    18     3     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [29]:
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'], max_seq_len, tokenizer)

  0%|          | 0/2490 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 2490/2490 [00:06<00:00, 363.22it/s]


In [30]:
# 최대 길이 128로 설정됨
# val_data 첫번째째 샘플 확인
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩 길이 :', len(input_id))
print('정수 인코딩 복원:', tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  3673   636  2116  1041  2371  2062    16     6  4122    16  1535
  1458 10283    18     6     3   636  2259  3741  4942  2116   636  2138
  4105  2223  2155  6000  4122  2170  2318  4117  2138   572  2359  2062
    18     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [31]:
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len, tokenizer)

  0%|          | 0/5010 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 5010/5010 [00:06<00:00, 764.13it/s]


In [32]:
# 최대 길이 128로 설정됨
# test_data 첫번째째 샘플 확인
input_id = X_test[0][0]
attention_mask = X_test[1][0]
token_type_id = X_test[2][0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩 길이 :', len(input_id))
print('정수 인코딩 복원:', tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [   2 8911   16  717 2259 3724 2170 9149 3628 2446 2232 1889 2118 1380
 2886 3683   16  717 2259 3760 8642 2371 2088   16 3983  636 2170 2318
 3690 3758 2205 2318  859 2359 2062   18    3  717 2259  636 2522 3690
 3758 2205 2118 1380 2886 2062   18    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [33]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

In [35]:
# 각각 고유한 정수부여 : contraction 0, entailment 1, neutral 2
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)

In [36]:
print(train_data['gold_label'][0], y_train[0])
print(val_data['gold_label'][0], y_val[0])
print(test_data['gold_label'][0], y_test[0])


contradiction 0
neutral 2
contradiction 0


### 3. 모델 정의

In [37]:
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels, 
                                            kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), 
                                            activation='softmax',
                                            name='classifier')
  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    cls_token = outputs[1]
    prediction = self.classifier(cls_token)
    return prediction

In [38]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [39]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [40]:
with strategy.scope():
  model = TFBertForSequenceClassification('klue/bert-base', num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [42]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=2)

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/5
29432/29432 [==============================] - 2504s 82ms/step - loss: 0.5327 - accuracy: 0.7881 - val_loss: 0.5422 - val_accuracy: 0.7863
Epoch 2/5
29432/29432 [==============================] - 2371s 81ms/step - loss: 0.4352 - accuracy: 0.8325 - val_loss: 0.5659 - val_accuracy: 0.7683
Epoch 3/5
29432/29432 [==============================] - 2362s 80ms/step - loss: 1.0555 - accuracy: 0.3658 - val_loss: 1.0991 - val_accuracy: 0.3333


In [43]:
print("\n 테스트 정확도 : %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)[1]))

5/5 [==============================] - 15s 2s/step - loss: 1.0991 - accuracy: 0.3333

 테스트 정확도 : 0.3333
